https://www.science.org/doi/10.1126/science.1127647

https://github.com/L1aoXingyu/pytorch-beginner/blob/9c86be785c7c318a09cf29112dd1f1a58613239b/08-AutoEncoder/conv_autoencoder.py

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn
from pathlib import Path
from torch.utils.data import TensorDataset, DataLoader
from lib.modules import (
    evaluate_loop, 
    pad_for_windowing,
    window_session,
    optimization_loop
)
from lib.models import LinearAutoencoder, ConvAutoencoder
from tqdm import tqdm
import plotly.express as px
from datetime import timedelta
from sklearn.model_selection import train_test_split

In [3]:
WINSIZE = 101
DEVICE = 'cuda:0'
RAW_DIR = Path('/home/musa/datasets/eating_raw/')

In [4]:
recordings = []
for rec_dir in RAW_DIR.iterdir():
    recordings.append(rec_dir)
    print(rec_dir.name)

11-01_20_34_28
10-27_00_21_25
11-01_20_54_52
10-27_09_45_42
11-02_19_28_19
10-28_13_18_42
10-27_00_20_15


In [5]:
accelerations = []
for session_dir in recordings:
    accel_file = session_dir / f'acceleration-{session_dir.name}.csv'
    acceleration = pd.read_csv(accel_file,skiprows=1).rename({'x': 'x_acc', 'y': 'y_acc', 'z': 'z_acc'}, axis=1)
    # acceleration = acceleration.dropna()

    acceleration_start_time_seconds = float(pd.read_csv(session_dir / accel_file, nrows=1,header=None).iloc[0,0].split()[-1])/1000
    acceleration.timestamp = ((acceleration.timestamp - acceleration.timestamp[0])*1e-9)+acceleration_start_time_seconds # get timestamp in seconds

    accelerations.append(acceleration)

    print(f'Index: {len(accelerations)-1}, Date: {session_dir.name}, nSamples: {len(acceleration)}, Time Elapsed: {timedelta(seconds=acceleration.timestamp.iloc[-1] - acceleration.timestamp.iloc[0])}')


Index: 0, Date: 11-01_20_34_28, nSamples: 127133, Time Elapsed: 0:20:21.474008
Index: 1, Date: 10-27_00_21_25, nSamples: 260457, Time Elapsed: 0:51:35.645162
Index: 2, Date: 11-01_20_54_52, nSamples: 4080424, Time Elapsed: 22:11:22.872910
Index: 3, Date: 10-27_09_45_42, nSamples: 2854125, Time Elapsed: 7:37:03.002257
Index: 4, Date: 11-02_19_28_19, nSamples: 3082769, Time Elapsed: 19:45:45.052016
Index: 5, Date: 10-28_13_18_42, nSamples: 19536627, Time Elapsed: 4 days, 5:58:05.118965
Index: 6, Date: 10-27_00_20_15, nSamples: 6919, Time Elapsed: 0:01:06.511446


In [ ]:
dim_amt = 5
i = 5
fig = px.line(accelerations[i][::dim_amt], x=accelerations[i].index[::dim_amt], y=['x_acc','y_acc','z_acc'])
fig.show(renderer='browser')

In [6]:
skip_idxs = [5]

train_idxs = [0,3]
Xtr = []
Xte = []

for i,accel in enumerate(accelerations):
    if i in skip_idxs:
        continue

    X = torch.Tensor(accel[['x_acc','y_acc','z_acc']].values)
    X = pad_for_windowing(X, WINSIZE)
    X = window_session(X, WINSIZE)
    print(X.shape)

    if i in train_idxs:
        Xte.append(X)
    else:
        Xtr.append(X)

Xtr = torch.cat(Xtr)
Xte = torch.cat(Xte)

trainloader = DataLoader(TensorDataset(Xtr), batch_size=64, shuffle=True)
testloader = DataLoader(TensorDataset(Xte), batch_size=64)

torch.Size([127133, 303])
torch.Size([260457, 303])
torch.Size([4080424, 303])
torch.Size([2854125, 303])
torch.Size([3082769, 303])
torch.Size([6919, 303])


In [25]:
model = ConvAutoencoder(winsize=WINSIZE).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.MSELoss()

In [26]:
losses = []
test_losses = []
pbar = tqdm(range(10))
for epoch in pbar:
    lossi = 0
    test_lossi = 0

    for X in trainloader:
        X = X[0].to(DEVICE)

        # Foward Pass
        logits = model(X)
        loss = criterion(logits, X)

        # Backwards Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Sum Loss
        lossi += loss.item()
    
    # Test model
    for X in testloader:
        X = X[0].to(DEVICE)
        logits = model(X)
        test_lossi += criterion(logits, X).item()

    losses.append(lossi/len(trainloader))
    test_losses.append(test_lossi/len(testloader))
    pbar.set_description(f'Epoch {epoch}: Train Loss: {losses[-1]:.5}, Test Loss: {test_losses[-1]:.5}')

    plt.plot(losses)
    plt.plot(test_losses)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: conv_transpose1d(): argument 'padding' (position 5) must be tuple of ints, but found element of type str at pos 0

In [11]:
torch.save(model.state_dict(), 'dev/autoencorder.pt')

In [7]:
model.load_state_dict(torch.load('dev/autoencorder.pt'))

<All keys matched successfully>

In [8]:
# Recreate a signal with trained model

acceleration = accelerations[3]

Xte = torch.Tensor(acceleration[['x_acc','y_acc','z_acc']].values)
Xte = pad_for_windowing(Xte, WINSIZE)
Xte = window_session(Xte, WINSIZE)

testloader = DataLoader(TensorDataset(Xte), batch_size=64)

Xpred = []
for X in tqdm(testloader):
    X = X[0].to(DEVICE)
    logits = model(X)
    Xpred.append(logits)

Xpred = torch.cat(Xpred)
Xpred = Xpred.view(-1,3,101)[:,:,50].T # unwindow

acceleration['x_pred'] = Xpred[0].cpu().detach()
acceleration['y_pred'] = Xpred[1].cpu().detach()
acceleration['z_pred'] = Xpred[2].cpu().detach()

100%|██████████| 44596/44596 [00:20<00:00, 2157.21it/s]


In [9]:
dim_amt = 10
fig = px.line(acceleration[::dim_amt], x=acceleration.index[::dim_amt], y=['x_acc','y_acc','z_acc', 'x_pred', 'y_pred', 'z_pred'])
fig.show(renderer='browser')